In [ ]:
pip install pandas-market-calendars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 11.3 MB/s eta 0:00:00


In [ ]:
pip install bs4

In [ ]:
pip install finviz

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for finviz: filename=finviz-1.4.6-py3-none-any.whl size=16673 sha256=4416b93afe22e1962839145e506b858c0e100ac20c49556aebdfdfdcf25a4ba6
  Stored in directory: /root/.cache/pip/wheels/0e/97/1b/dd5977a878c00130a0d4f585c118a394eb62be8640d54e0ef0
  Created wheel for user_agent: filename=user_agent-0.1.10-py3-none-any.whl size=18967 sha256=3a0a17761db94843341f798cc1d836468c10c6309161fd4563cd09304240ee28
  Stored in directory: /root/.cache/pip/wheels/69/29/26/1956a891a058037774285ee79ab5c3ecf034dba50a4198fedd
Successfully built finviz user_agent


In [ ]:
pip install finvizfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from finvizfinance.screener.overview import Overview

class StockScreener:
    def __init__(self, multiplicador_volumen=3.3):
        self.multiplicador_volumen = multiplicador_volumen

    def calcular_variacion_porcentual(self, serie, periodos):
        """
        Calcula la variación porcentual de manera segura
        """
        try:
            if len(serie) >= periodos:
                return ((serie.iloc[-1].item() / serie.iloc[-periodos].item() - 1) * 100)
            return 0.0
        except Exception:
            return 0.0

    def verificar_criterios(self, ultimo_precio, ultimo_volumen, volumen_promedio, ma20, ma50):
        """
        Verifica los criterios técnicos
        """
        try:
            # Asegurar que estamos trabajando con valores escalares
            vol = float(ultimo_volumen if isinstance(ultimo_volumen, (int, float)) else ultimo_volumen.item())
            vol_prom = float(volumen_promedio if isinstance(volumen_promedio, (int, float)) else volumen_promedio.item())
            ma20_val = float(ma20 if isinstance(ma20, (int, float)) else ma20.item())
            ma50_val = float(ma50 if isinstance(ma50, (int, float)) else ma50.item())

            # Realizar comparaciones con valores escalares
            volumen_anomalo = vol >= (vol_prom * self.multiplicador_volumen)
            tendencia_alcista = ma20_val > ma50_val

            return volumen_anomalo and tendencia_alcista
        except Exception as e:
            print(f"Error en verificación de criterios: {str(e)}")
            return False

    def analizar_stock(self, symbol):
        """
        Analiza un stock individual
        """
        try:
            # Obtener datos
            end_date = datetime.now()
            start_date = end_date - timedelta(days=100)
            stock = yf.download(symbol, start=start_date, end=end_date, progress=False)

            if len(stock) < 50:
                return None

            try:
                # Medias móviles
                stock['MA20'] = stock['Close'].rolling(window=20).mean()
                stock['MA50'] = stock['Close'].rolling(window=50).mean()
                stock['MA200'] = stock['Close'].rolling(window=200).mean()

                # RSI
                delta = stock['Close'].diff()
                gain = delta.clip(lower=0)
                loss = -delta.clip(upper=0)
                avg_gain = gain.rolling(window=14).mean()
                avg_loss = loss.rolling(window=14).mean()
                rs = avg_gain / avg_loss
                stock['RSI'] = 100 - (100 / (1 + rs))

                # MACD
                exp1 = stock['Close'].ewm(span=12, adjust=False).mean()
                exp2 = stock['Close'].ewm(span=26, adjust=False).mean()
                stock['MACD'] = exp1 - exp2
                stock['Signal'] = stock['MACD'].ewm(span=9, adjust=False).mean()

                # Obtener últimos valores (asegurando que son escalares)
                ultimo_precio = stock['Close'].iloc[-1].item()
                ultimo_volumen = stock['Volume'].iloc[-1].item()
                volumen_promedio = stock['Volume'].rolling(window=20).mean().iloc[-1].item()
                ma20_ultimo = stock['MA20'].iloc[-1].item()
                ma50_ultimo = stock['MA50'].iloc[-1].item()

                # Verificar criterios
                if self.verificar_criterios(ultimo_precio, ultimo_volumen, volumen_promedio, ma20_ultimo, ma50_ultimo):
                    return {
                        'Símbolo': symbol,
                        'Precio': round(ultimo_precio, 2),
                        'Volumen': int(ultimo_volumen),
                        'Volumen Promedio': int(volumen_promedio),
                        'Mult. Volumen': round(ultimo_volumen/volumen_promedio, 2),
                        'MA20': round(ma20_ultimo, 2),
                        'MA50': round(ma50_ultimo, 2),
                        'MA200': round(stock['MA200'].iloc[-1].item(), 2) if len(stock) >= 200 else 0,
                        'RSI': round(stock['RSI'].iloc[-1].item(), 2),
                        'MACD': round(stock['MACD'].iloc[-1].item(), 3),
                        'Señal MACD': round(stock['Signal'].iloc[-1].item(), 3),
                        'Var % 5d': round(self.calcular_variacion_porcentual(stock['Close'], 5), 2),
                        'Var % 20d': round(self.calcular_variacion_porcentual(stock['Close'], 20), 2),
                        'Vol % 5d': round(self.calcular_variacion_porcentual(stock['Volume'], 5), 2)
                    }
                return None

            except Exception as e:
                print(f"Error en cálculos técnicos para {symbol}: {str(e)}")
                return None

        except Exception as e:
            print(f"Error descargando datos para {symbol}: {str(e)}")
            return None

    def obtener_simbolos_finviz(self):
        """
        Obtiene símbolos usando finvizfinance
        """
        try:
            foverview = Overview()
            all_stocks = pd.DataFrame()

            configuraciones = [
                # Micro caps
                {
                    'price_range': '$1 to $5',
                    'market_cap': 'Micro ($50mln to $300mln)'
                },
                {
                    'price_range': '$5 to $10',
                    'market_cap': 'Micro ($50mln to $300mln)'
                },
                {
                    'price_range': '$10 to $20',
                    'market_cap': 'Micro ($50mln to $300mln)'
                },
                # Small caps
                {
                    'price_range': '$1 to $5',
                    'market_cap': 'Small ($300mln to $2bln)'
                },
                {
                    'price_range': '$5 to $10',
                    'market_cap': 'Small ($300mln to $2bln)'
                },
                {
                    'price_range': '$10 to $20',
                    'market_cap': 'Small ($300mln to $2bln)'
                },
                {
                    'price_range': '$20 to $50',
                    'market_cap': 'Small ($300mln to $2bln)'
                }
            ]

            exchanges = ['NASDAQ', 'NYSE', 'AMEX']

            for config in configuraciones:
                for exchange in exchanges:
                    print(f"Buscando en {exchange} - Precio: {config['price_range']}, Cap: {config['market_cap']}...")
                    filters_dict = {
                        'Price': config['price_range'],
                        'Market Cap.': config['market_cap'],
                        'Average Volume': 'Over 100K',
                        'Exchange': exchange,
                        'Country': 'USA'
                    }

                    try:
                        foverview.set_filter(filters_dict=filters_dict)
                        df = foverview.screener_view()
                        if df is not None and not df.empty:
                            all_stocks = pd.concat([all_stocks, df], ignore_index=True)
                            print(f"✓ Encontrados {len(df)} stocks")
                        time.sleep(1)
                    except Exception as e:
                        print(f"Error en {exchange}: {str(e)}")
                        continue

            if not all_stocks.empty:
                unique_stocks = all_stocks['Ticker'].unique().tolist()
                print(f"\nTotal de stocks únicos encontrados: {len(unique_stocks)}")
                return unique_stocks
            return []

        except Exception as e:
            print(f"Error general: {str(e)}")
            return []

    def ejecutar_screening(self):
        """
        Ejecuta el proceso completo de screening
        """
        try:
            symbols = self.obtener_simbolos_finviz()
            print(f"\nAnalizando {len(symbols)} símbolos...")

            resultados = []
            stocks_error = []

            for i, symbol in enumerate(symbols, 1):
                try:
                    if i % 10 == 0:
                        print(f"Progreso: {i}/{len(symbols)} ({round(i/len(symbols)*100, 1)}%)")
                    resultado = self.analizar_stock(symbol)
                    if resultado:
                        resultados.append(resultado)
                        print(f"✓ {symbol}: ${resultado['Precio']} | Vol: {resultado['Mult. Volumen']}x | RSI: {resultado['RSI']}")
                    time.sleep(0.2)
                except KeyboardInterrupt:
                    print("\n\nInterrumpido por el usuario. Guardando resultados parciales...")
                    break
                except Exception as e:
                    stocks_error.append(symbol)
                    print(f"Error completo en {symbol}: {str(e)}")

            if stocks_error:
                print(f"\nStocks con error: {len(stocks_error)}")
                print("Símbolos:", ", ".join(stocks_error))

            df_resultados = pd.DataFrame(resultados)
            if len(df_resultados) > 0:
                return df_resultados.sort_values(['Mult. Volumen', 'Var % 5d'], ascending=[False, False])
            return pd.DataFrame()

        except Exception as e:
            print(f"Error en screening: {str(e)}")
            return pd.DataFrame()

    def guardar_resultados(self, df, filename='stocks_screening.csv'):
        """
        Guarda los resultados en CSV y muestra un resumen
        """
        if not df.empty:
            df.to_csv(filename, index=False)
            print(f"\nResultados guardados en {filename}")
            print("\nResumen:")
            print(f"Total stocks: {len(df)}")
            print(f"Rango de precios: ${df['Precio'].min():.2f} - ${df['Precio'].max():.2f}")
            print(f"Multiplicador de volumen promedio: {df['Mult. Volumen'].mean():.2f}x")
            print(f"RSI promedio: {df['RSI'].mean():.2f}")
            print(f"Variación % 5d promedio: {df['Var % 5d'].mean():.2f}%")
            print("\nTop 5 por volumen:")
            print(df[['Símbolo', 'Precio', 'Mult. Volumen', 'RSI', 'Var % 5d']].head())

def main():
    try:
        screener = StockScreener(multiplicador_volumen=3.3)
        print("Iniciando screening de stocks...")
        resultados = screener.ejecutar_screening()

        if not resultados.empty:
            print("\nStocks encontrados que cumplen los criterios:")
            print(resultados)
            screener.guardar_resultados(resultados)
        else:
            print("\nNo se encontraron stocks que cumplan los criterios.")

    except KeyboardInterrupt:
        print("\nPrograma interrumpido por el usuario")
    except Exception as e:
        print(f"Error en la ejecución: {str(e)}")

if __name__ == "__main__":
    main()

Iniciando screening de stocks...
Buscando en NASDAQ - Precio: $1 to $5, Cap: Micro ($50mln to $300mln)...


/usr/local/lib/python3.10/dist-packages/finvizfinance/screener/base.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame(frame)], ignore_index=True)


✓ Encontrados 190 stocks
Buscando en NYSE - Precio: $1 to $5, Cap: Micro ($50mln to $300mln)...
✓ Encontrados 39 stocks
Buscando en AMEX - Precio: $1 to $5, Cap: Micro ($50mln to $300mln)...
✓ Encontrados 12 stocks
Buscando en NASDAQ - Precio: $5 to $10, Cap: Micro ($50mln to $300mln)...
✓ Encontrados 65 stocks
Buscando en NYSE - Precio: $5 to $10, Cap: Micro ($50mln to $300mln)...
✓ Encontrados 32 stocks
Buscando en AMEX - Precio: $5 to $10, Cap: Micro ($50mln to $300mln)...
✓ Encontrados 3 stocks
Buscando en NASDAQ - Precio: $10 to $20, Cap: Micro ($50mln to $300mln)...
✓ Encontrados 35 stocks
Buscando en NYSE - Precio: $10 to $20, Cap: Micro ($50mln to $300mln)...
✓ Encontrados 7 stocks
Buscando en AMEX - Precio: $10 to $20, Cap: Micro ($50mln to $300mln)...
✓ Encontrados 3 stocks
Buscando en NASDAQ - Precio: $1 to $5, Cap: Small ($300mln to $2bln)...
✓ Encontrados 70 stocks
Buscando en NYSE - Precio: $1 to $5, Cap: Small ($300mln to $2bln)...
✓ Encontrados 52 stocks
Buscando en AME

<ipython-input-26-515d511f549c>:170: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_stocks = pd.concat([all_stocks, df], ignore_index=True)


Buscando en NASDAQ - Precio: $5 to $10, Cap: Small ($300mln to $2bln)...
✓ Encontrados 131 stocks
Buscando en NYSE - Precio: $5 to $10, Cap: Small ($300mln to $2bln)...
✓ Encontrados 110 stocks
Buscando en AMEX - Precio: $5 to $10, Cap: Small ($300mln to $2bln)...
✓ Encontrados 3 stocks
Buscando en NASDAQ - Precio: $10 to $20, Cap: Small ($300mln to $2bln)...
✓ Encontrados 170 stocks
Buscando en NYSE - Precio: $10 to $20, Cap: Small ($300mln to $2bln)...
✓ Encontrados 184 stocks
Buscando en AMEX - Precio: $10 to $20, Cap: Small ($300mln to $2bln)...
✓ Encontrados 4 stocks
Buscando en NASDAQ - Precio: $20 to $50, Cap: Small ($300mln to $2bln)...
✓ Encontrados 142 stocks
Buscando en NYSE - Precio: $20 to $50, Cap: Small ($300mln to $2bln)...
✓ Encontrados 97 stocks
Buscando en AMEX - Precio: $20 to $50, Cap: Small ($300mln to $2bln)...
No ticker found.

Total de stocks únicos encontrados: 1352

Analizando 1352 símbolos...
Progreso: 10/1352 (0.7%)
Progreso: 20/1352 (1.5%)
✓ BCOV: $2.14 | 

ERROR:yfinance:Could not get exchangeTimezoneName for ticker 'SITC' reason: 'chart'
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SITC']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Progreso: 1080/1352 (79.9%)
Progreso: 1090/1352 (80.6%)
Progreso: 1100/1352 (81.4%)
Progreso: 1110/1352 (82.1%)
Progreso: 1120/1352 (82.8%)
✓ ATSG: $22.0 | Vol: 13.31x | RSI: 91.46
Progreso: 1130/1352 (83.6%)
Progreso: 1140/1352 (84.3%)
Progreso: 1150/1352 (85.1%)
Progreso: 1160/1352 (85.8%)
Progreso: 1170/1352 (86.5%)
✓ IART: $24.12 | Vol: 3.87x | RSI: 78.28
Progreso: 1180/1352 (87.3%)
Progreso: 1190/1352 (88.0%)
Progreso: 1200/1352 (88.8%)
Progreso: 1210/1352 (89.5%)
Progreso: 1220/1352 (90.2%)
Progreso: 1230/1352 (91.0%)
Progreso: 1240/1352 (91.7%)
Progreso: 1250/1352 (92.5%)
Progreso: 1260/1352 (93.2%)
Progreso: 1270/1352 (93.9%)
Progreso: 1280/1352 (94.7%)
Progreso: 1290/1352 (95.4%)
Progreso: 1300/1352 (96.2%)
Progreso: 1310/1352 (96.9%)
Progreso: 1320/1352 (97.6%)
Progreso: 1330/1352 (98.4%)
Progreso: 1340/1352 (99.1%)
Progreso: 1350/1352 (99.9%)

Stocks encontrados que cumplen los criterios:
  Símbolo  Precio   Volumen  Volumen Promedio  Mult. Volumen   MA20   MA50  \
4     CEP